In [2]:
import tweepy
import json
from itertools import islice
import os
import time
import pprint
import requests
import random
import networkx as nx
from networkx.algorithms.approximation import clique
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network
import itertools 
from scipy import stats

print("Librerie caricate!")


Librerie caricate!


In [3]:
data_folder = "data_ids"
pp=pprint.PrettyPrinter()

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [3]:
#Fase di autenticazione
api_key="cvoM8D7hXXxlvBXTM8aH9X2ec"
api_secret="Uk2UvH0FJY7KaDzkXYLfgiYkA1OuCwbLlGFPyodB5wcQ5bsItN"
access_token="3303466053-OLuExo5KcP8UQCVwZwmyakZs8b91Fpl2lMOUDAe"
access_secret="1lMXufu42KN8JvJjYT7c0zI3Q57CkN09BxkNXZuNQ0Dej"
bearer_token="AAAAAAAAAAAAAAAAAAAAAHsVJQEAAAAAQ4vYb83r6ueD8QvjJ4Zpx9R7Kbw%3DQuLzsmDYOvpff7lRHGXhNJSOXTFuPyOwLHZv7HPSj9WF34h1E8"

auth=tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print("Auth success")

Auth success


In [4]:
users_screen_name=["mizzaro","damiano10","miccighel_","eglu81","KevinRoitero"]
users_id=[18932422, 132646210, 15750573, 19659370, 3036907250]

In [ ]:
#1. Scaricare gli id dei followers e dei following dei 5 account principali
followers_ids,following_ids={},{}

for utente in users_id:
    print(utente)
    followers_utente,following_utente=[],[]
    for item in tweepy.Cursor(
        api.followers,
        id=utente,
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        user={"id":json_data["id"]}
        time.sleep(181)
        followers_utente.append(user)

    followers_ids[utente]=followers_utente
    

    for item in tweepy.Cursor(
        api.friends,
        id=utente,
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        user={"id":json_data["id"]}
        time.sleep(181)
        following_utente.append(user)

    following_ids[utente]=following_utente

print(following_ids)

#export to json
serialize_json("data_ids","followers_5_utenti.json",followers_ids)
serialize_json("data_ids","following_5_utenti.json",following_ids)

In [ ]:
followers_ids, following_ids = [], []

# punto 2&3
followers_ids = read_json("data_ids/followers_5_utenti.json")
following_ids = read_json("data_ids/following_5_utenti.json")

# 2
random_followers_utenti_ids, followers_of_followers_ids = {}, {}

for utente in followers_ids:
    random_followers_utenti_ids[utente] = random.sample(followers_ids[utente], 5)
# result: random_followers_utenti_ids= {"mizzaro":[{}{}{}{}{}],"damiano10":[{}...}],...}

for utente in following_ids:
    random_following_utenti_ids[utente] = random.sample(following_ids[utente], 5)
# result: random_following_utenti_ids= {"mizzaro":[{}{}{}{}{}],"damiano10":[{}...}],...}

#Download dei followers dei 10 followers degli utenti random 
try:
    for utente in random_followers_utenti_ids:
        for f in random_followers_utenti_ids[utente]:
            fof = []
            print("scarico per " + str(utente))
            for item in tweepy.Cursor(
                    api.followers,
                    id=f['id'],
                    skip_status=True,
                    include_user_entities=False
            ).items(10):
                    time.sleep(10)
                    json_data = item._json
                    user = {"id": json_data["id"]}
                    fof.append(user)
                    print("Downloaded: " + str(user))


            followers_of_followers_ids[f['id']] = fof
except tweepy.TweepError as error: 
    print(error)
serialize_json("data_ids/finale", "followers_of_followers.json", followers_of_followers_ids)

#Download dei following dei 10 following degli utenti random 
try:
    for utente in random_following_utenti_ids:
        for f in random_following_utenti_ids[utente]:
            fof = []
            print("scarico per " + str(utente))
            for item in tweepy.Cursor(
                    api.friends,
                    id=f['id'],
                    skip_status=True,
                    include_user_entities=False
            ).items(10):
                time.sleep(10)
                json_data = item._json
                user = {"id": json_data["id"]}
                fof.append(user)
                print("Downloaded: " + str(user))

            following_of_following_ids[f['id']] = fof
except tweepy.TweepError as error: 
    print(error)
serialize_json("data_ids/finale", "following_of_following.json", following_of_following_ids)

In [5]:
#Unione di tutti gli id rilevati e rimozione dei duplicati

#Caricamento json contenente tutti gli ID dei nodi interessati
followers_ids=read_json("data_ids/followers_5_utenti.json")
following_ids=read_json("data_ids/following_5_utenti.json")
followers_of_followers_ids=read_json("data_ids/followers_of_followers.json")
following_of_following_ids=read_json("data_ids/following_of_following.json")

print("Json caricati")

lista_json=[followers_ids,following_ids, followers_of_followers_ids,following_of_following_ids]

id_nodi_grafo=[]
#Eliminazione duplicati e conteggio di essi.
for jsonn in lista_json:
    for user in jsonn:
        for user_id in jsonn[user]:
            if not user_id in id_nodi_grafo:
                id_nodi_grafo.append(user_id)

print(len(id_nodi_grafo))


Data read from path: data_ids/followers_5_utenti.json
Data read from path: data_ids/following_5_utenti.json
Data read from path: data_ids/followers_of_followers.json
Data read from path: data_ids/following_of_following.json
Json caricati
3103


In [ ]:
#4 Scaricare i dati di ogni utente con api.get_user 

nodes = {}
    
for user_id in id_nodi_grafo: 
    #richiamo API GET_USER
    utente=api.get_user(id=user_id['id'])._json
    node_infos= {} 
    
    node_infos["name"]=utente["name"]
    node_infos["screen_name"]=utente["screen_name"]
    node_infos["location"]=utente["location"]
    node_infos["followers_count"]=utente["followers_count"]
    node_infos["friends_count"]=utente["friends_count"]
    node_infos["statuses_count"]=utente["statuses_count"]
    node_infos["created_at"]=utente["created_at"]
   
    nodes[user_id['id']] = node_infos
print(nodes)
print("Download dei nodi effettuato")
serialize_json("data_ids", "nodes_of_twitter_graph.json", nodes)

In [9]:
#5 Controllo relazioni di following e follow con la api.show_friendship
# Questo è il codice ideale per scaricare con un'unica procedura tutte le relazioni utente - main_user,  
# ma realmente è stato modificato il codice per eseguirlo parallelamente per ogni singolo utente da account twitter developper diversi
friendships=[]

print("json letto")
contatore=0
for main_user in user_id:
    for node_id in id_nodi_grafo:
        print(contatore/len(id_nodi_grafo))
        time.sleep(1)
        try:
            #relationship = api.show_friendship(source_id=node_id['id'],target_id=main_user)
            relation = relationship[0]
            #Controllo nell'oggetto relationship delle relazioni di follow e following
            if(relation.following == True): # node follows the user
                infos_of_relation={}
                infos_of_relation["source"] = node_id['id']
                infos_of_relation["type"] = "following"
                infos_of_relation["target"] = main_user
                friendships.append(infos_of_relation)
            if(relation.followed_by == True): # user follows the node
                infos_of_relation={}
                infos_of_relation["source"] = main_user
                infos_of_relation["type"] = "follows"
                infos_of_relation["target"] = node_id['id']
                friendships.append(infos_of_relation)
        except:
            print(node_id['id'])
        contatore+=1
    

serialize_json("data_ids", "edges_of_twitter_graph.json", friendships)
print("Completamento controllo relazioni terminato")

json letto
0.0
740410382
0.0003222687721559781
1972411447
0.0006445375443119562
257691637
0.0009668063164679343
4893681
0.0012890750886239124
116897811
0.0016113438607798904


KeyboardInterrupt: 

In [8]:
# Unione delle relazioni scaricate per ogni utente, necessario per parallelizzare il download
damiano_edges=read_json("data_ids/api_show_friendship/damiano_edges.json")
eglu_edges=read_json("data_ids/api_show_friendship/eglu_edges.json")
kevin_edges=read_json("data_ids/api_show_friendship/kevin_edges.json")
micch_edges=read_json("data_ids/api_show_friendship/micch_edges.json")
mizzaro_edges=read_json("data_ids/api_show_friendship/mizzaro_edges.json")

lista_edges=[damiano_edges,eglu_edges, kevin_edges,micch_edges,mizzaro_edges]

main_edges = []
#Unione json
for main in lista_edges:
    for relation in main:
        main_edges.append(relation)
serialize_json("data_ids", "edges_of_twitter_graph.json", main_edges)

Data read from path: data_ids/api_show_friendship/damiano_edges.json
Data read from path: data_ids/api_show_friendship/eglu_edges.json
Data read from path: data_ids/api_show_friendship/kevin_edges.json
Data read from path: data_ids/api_show_friendship/micch_edges.json
Data read from path: data_ids/api_show_friendship/mizzaro_edges.json
Data serialized to path: data_ids/edges_of_twitter_graph.json


In [10]:
#5.2 Creazione del grafo
twitter_graph = nx.DiGraph(team="Loris Parata 144338, Francesco Arzon 142439, Lorenzo Dal Fabbro, Matteo Galvan")
#Aggiunta dei nodi al grafo
nodes_of_graph=read_json("data_ids/nodes_of_twitter_graph.json")
for ids, node in nodes_of_graph.items():
    twitter_graph.add_node(ids,
                               id= ids,
                               title= node["name"],
                               color ="#ffff00",
                               physics=False,#rende la visualizzazone del grafo più leggera
                               name=node['name'],
                               screen_name=node['screen_name'],
                               location=node['location'],
                               followers_count=node["followers_count"],
                               following_count=node["friends_count"],
                               number_of_twitts=node["statuses_count"],
                               data_creazione_profilo=node["created_at"]
                              ) 
#Aggiunta degli archi al grafo, con controllo se è presente nel grafo il nodo source, per rilevare eventuali incongurenze
for edge in main_edges:
    if edge['type'] == 'following': #non follows, a causa dell'inversione source - targhet nella funzione di show_frienship
        if twitter_graph.has_node(str(edge['source'])):
            twitter_graph.add_edge(str(edge['source']),str(edge['target']))
        else:
            print("Relazione relativa ad un nodo non presente nel grafo")
            
print(twitter_graph.number_of_nodes())
print(twitter_graph.number_of_edges())

Data read from path: data_ids/nodes_of_twitter_graph.json
3103
1922


In [8]:
#6 Creazione grafo interattivo con pyvis

def disegna_grafo(grafo):
    nt = Network(
        height ="80%",
        width = "80%",
        bgcolor="#222222",
        font_color="white",
        heading= grafo,
        directed=True,
    )
    nt.show_buttons(filter_=None)
    nt.from_nx(grafo)
   # nt.barnes_hut()
    nt.inherit_edge_colors(False)
  # nt.set_edge_smooth("continuous") #cambia formato di visualizzazione degli archi
    neighbor_map = nt.get_adj_list()
    
    for node in nt.nodes:
            node["value"] = len(neighbor_map[node["id"]])
           # print(str(node["id"])+":"+ str(node["value"]))
         
    nt.show("grafo.html")

disegna_grafo(twitter_graph)

In [11]:
#Rimozione dei nodi sconnessi per ottenere un sotto grafo connesso
print("Numero nodi del grafo completo")
print(twitter_graph.number_of_nodes())
print("Numero archi del grafo completo")
print(twitter_graph.number_of_edges())

#Creo una copia del grafo
sub_twitter_graph =twitter_graph.copy()

nodes_to_delete=[]
#Controlla se un nodo ha out_degree = 0,
#perchè ci interessano solo i nodi che seguono un account principale
for node_id in twitter_graph.nodes():
    if(twitter_graph.out_degree[node_id] == 0):
        nodes_to_delete.append(node_id)
sub_twitter_graph.remove_nodes_from(nodes_to_delete)

print("Numero di nodi eliminati")        
print(len(nodes_to_delete))
print("Numero dei nodi del grafo connesso")
print(sub_twitter_graph.number_of_nodes())

#disegna_grafo(sub_twitter_graph)

Numero nodi del grafo completo
3103
Numero archi del grafo completo
1922
Numero di nodi eliminati
1424
Numero dei nodi del grafo connesso
1679


In [12]:
#7 Proprietà del grafo completo
undirected_twitter_graph= nx.to_undirected(twitter_graph)

if nx.is_connected(undirected_twitter_graph):
    print("Il grafo è connesso.")
else:
    print("Il grafo non è connesso.")
    
if nx.is_bipartite(undirected_twitter_graph):
    print("Il grafo è bipartito.")
else:
    print("Il grafo non è bipartito.")
    

Il grafo non è connesso.
Il grafo non è bipartito.


In [13]:
#7.1 Proprietà del grafo connesso
undirected_sub_twitter_graph= nx.to_undirected(sub_twitter_graph)
if nx.is_connected(undirected_sub_twitter_graph):
    print("Il grafo è connesso.")
else:
    print("Il grafo non è connesso.")
    
if nx.is_bipartite(undirected_sub_twitter_graph):
    print("Il grafo è bipartito.")
else:
    print("Il grafo non è bipartito.")

Il grafo è connesso.
Il grafo non è bipartito.


In [24]:
#8 Distanze sul grafo connesso, ma unidirected, perchè non è fortemente connesso
centro = nx.center(undirected_sub_twitter_graph)
print(centro)
diametro = nx.diameter(undirected_sub_twitter_graph)
print(diametro)
raggio = nx.radius(undirected_sub_twitter_graph)
print(raggio)

['3036907250', '19659370', '132646210']
4
2


In [27]:
#9 Misure di centralità sul grafo completo
#btw_centrality = nx.betweenness_centrality(twitter_graph)
#print("")
#btw_centrality=sorted(btw_centrality.items(),key=lambda x:x[1],reverse=True)
#print(btw_centrality)
#cls_centrality = nx.closeness_centrality(sub_twitter_graph)
#cls_centrality = sorted(cls_centrality.items(),key=lambda x:x[1],reverse=True)
#print(cls_centrality)
#dg_centrality = nx.degree_centrality(twitter_graph)
#dg_centrality=sorted(dg_centrality.items(),key=lambda x:x[1],reverse=True)
#print(dg_centrality)

#in_centrality= nx.in_degree_centrality(twitter_graph)
#in_centrality=sorted(in_centrality.items(),key=lambda x:x[1],reverse=True)
#print(in_centrality)
#out_centrality= nx.out_degree_centrality(twitter_graph)
#print(out_centrality)
#valori di centralità
#undirect_centralities=[btw_centrality,cls_centrality,dg_centrality]
#direct_centralities=[in_centrality,out_centrality]

#pagerank= nx.pagerank(twitter_graph)
#pagerank=sorted(pagerank.items(),key=lambda x:x[1],reverse=True)
#centralities= [btw_centrality,cls_centrality,dg_centrality,in_centrality,out_centrality]
#print(pagerank[:5])

#hits= nx.hits(twitter_graph)
#hits_hubs=hits[0]
#hits_hub= sorted(hits_hubs.items(),key=lambda x:x[1],reverse=True)
#print(hits_hub[:5])
#hits_authorities=hits[1]
#hits_authorities=sorted(hits_authorities.items(),key=lambda x:x[1],reverse=True)
#print(hits_authorities[:5])

[('132646210', 0.25499677627337203), ('19659370', 0.17504835589941972), ('15750573', 0.10863958736299162), ('18932422', 0.05190199871050935), ('3036907250', 0.03417150225660864), ('94732055', 0.0012894906511927789), ('14451127', 0.0012894906511927789), ('810893744593584128', 0.0012894906511927789), ('125483940', 0.0012894906511927789), ('2190533245', 0.0012894906511927789), ('2309104237', 0.0012894906511927789), ('341000847', 0.0012894906511927789), ('338989477', 0.0012894906511927789), ('249661913', 0.0012894906511927789), ('2601589496', 0.0012894906511927789), ('15455450', 0.0012894906511927789), ('130671142', 0.0012894906511927789), ('16868154', 0.0012894906511927789), ('78969930', 0.0012894906511927789), ('393675585', 0.0012894906511927789), ('1556357336', 0.0012894906511927789), ('116897811', 0.0009671179883945841), ('514667085', 0.0009671179883945841), ('920700510306529280', 0.0009671179883945841), ('923649456981135367', 0.0009671179883945841), ('890413838578966528', 0.0009671179

In [21]:
#9 Misure di centralità sul sotto-grafo
btw_centrality = nx.betweenness_centrality(sub_twitter_graph)
#print(btw_centrality)
btw_centrality=sorted(btw_centrality.items(),key=lambda x:x[1],reverse=True)
print(btw_centrality)


[('132646210', 0.0012819802089974392), ('19659370', 0.0006506145805422351), ('15750573', 0.0004623894429033436), ('18932422', 0.0001937332993130315), ('3036907250', 0.00015008259873409434), ('740410382', 0.0), ('1972411447', 0.0), ('257691637', 0.0), ('4893681', 0.0), ('116897811', 0.0), ('551951577', 0.0), ('91697719', 0.0), ('94732055', 0.0), ('514667085', 0.0), ('4149032843', 0.0), ('4188048952', 0.0), ('14451127', 0.0), ('2988561009', 0.0), ('920700510306529280', 0.0), ('950377242483343360', 0.0), ('1527607790', 0.0), ('923649456981135367', 0.0), ('890413838578966528', 0.0), ('1477918760', 0.0), ('810893744593584128', 0.0), ('125483940', 0.0), ('406303880', 0.0), ('4863093417', 0.0), ('2924886188', 0.0), ('2190533245', 0.0), ('3095317209', 0.0), ('153764165', 0.0), ('748315519674048512', 0.0), ('386908329', 0.0), ('21192987', 0.0), ('2826508686', 0.0), ('65915718', 0.0), ('203070657', 0.0), ('3916694415', 0.0), ('3761306416', 0.0), ('384127183', 0.0), ('3513346942', 0.0), ('1150918

In [ ]:
#Sottografo del nodo KevinRoitero
sub_graph_KevinRoitero = nx.ego_graph(twitter_graph, "3036907250")
disegna_grafo(sub_graph_KevinRoitero)
print(clique.max_clique(sub_graph_KevinRoitero))


In [ ]:
#11 copertua minima degli archi
nx.min_edge_cover(undirected_twitter_graph)


In [1]:
#12 Small-world-ness
omega=nx.omega(undirected_twitter_graph, niter=10, nrand=2)
print(omega)
sigma=nx.sigma(undirected_twitter_graph,niter=10, nrand=2)
print(sigma)

NameError: name 'omega' is not defined

In [ ]:
#13 Correlazione di Pearson e di Kendall

columns=["name","value"]
centralita_df = pd.DataFrame(columns=columns)
rows = [["btw_centrality",btw_centrality],["cls_centrality",cls_centrality],["dg_centrality",dg_centrality],["in_centrality",in_centrality],["out_centrality",out_centrality]]
#Creazione df contenente tipologia_misura : valore
for row in rows:
    centralita_df.loc[len(centralita_df)] = row
#print(centralita_df)

#Creazione df contenente  tipologia_misura_1 , tipologia_misura_2, valore_correlazione_1 , valore_correlazione_2
columns=["name_1","name_2","pearson", "kendall"]
result= pd.DataFrame(columns=columns)

for index, param_1 in centralita_df.iterrows():
    for index, param_2 in centralita_df.iterrows():
        if(param_1["name"] != param_2["name"]):
            name_1= param_1["name"]
            name_2= param_2["name"]
            array_1= np.array(list(param_1["value"].values()), dtype=float)
            array_2= np.array(list(param_2["value"].values()), dtype=float)
            pearson= stats.pearsonr(array_1,array_2)
            kendall= stats.kendalltau(array_1,array_2)
            new_row = {
            'name_1': name_1,
            'name_2': name_2,
            'pearson': pearson,
            'kendall': kendall
                }
            result = result.append(new_row, ignore_index=True)

result